# Coroutines
Coroutines are functions that can be paused and resumed at a particular point. These points/checkpoints are denoted with the keyword **yield** dividing the function into many parts. 
In order to resume from a particular point/checkpoint we use the keyword **next** .
When we use **next** and there is not a yield statement we obtain an ***StopIteration*** exception.
Furthermore, we can send values to the coroutine that stopped at a specific checkpoint using the keywork **send**.

In [1]:
def foo():
    print("This is the beginning of the function")
    yield
    print("Second part of the function")
    yield
    print("Final part of the function")

try:
    y = foo()
    print(type(y))
    next(y)
    next(y)
    next(y)
except StopIteration:
    print("StopIteration")

<class 'generator'>
This is the beginning of the function
Second part of the function
Final part of the function
StopIteration


In each checkpoint we can return the value of variable in that point

In [2]:
def bar():
    x = 1
    yield x
    x += 5
    yield x
    x -= 2
    yield x

try:
    y = bar()
    print(type(y))
    print(next(y))
    print(next(y))
    print(next(y))
except StopIteration:
    print("StopIteration")
    

<class 'generator'>
1
6
4


We can send values to the coroutine

In [3]:
def func():
    print("Starting function")

    x = yield
    print(x)
    print("Second part")

    a = yield
    print(a)
    print("Final part")

try:
    y = func()
    next(y)
    y.send(6)
    y.send(12)
except StopIteration as e:
    pass

# We need to use send when we are in a yield checkpoint, to reach this checkpoint we use send
try:
    print("We need to stop in a yield to send a value")
    y = func()
    # next(y)
    y.send(6)
    y.send(12)
except TypeError as e:
    print(e)

Starting function
6
Second part
12
Final part
We need to stop in a yield to send a value
can't send non-None value to a just-started generator


We can close a coroutine, it raises a ***GeneratorExit*** exception

In [4]:
def greet_name():
    try:
        while True:
            name = yield
            print("Hello", name)
    except GeneratorExit:
        print("Closing coroutine...")

y = greet_name()
y.__next__()
y.send("Juan")
y.send("Sebastian")
y.close()

Hello Juan
Hello Sebastian
Closing coroutine...


Knowing that coroutines can stop, resume and get values. We can use several coroutines to form a pipeline where there is not a main function that call the other, in this pipeline one coroutine can receive data, process it and sent it to other coroutine to visualization.

They seem simmilar to threads but they are not the same. Threads are managed by the operative system while coroutines are suspended and resumed by the programmer.

In [5]:
def word_producer(sentence, next_coroutines):
    words = sentence.split()
    for word in words:
        for next_coroutine in next_coroutines:
            next_coroutine.send(word)
    next_coroutine.close()

def filter_match_character(char=",", next_coroutine=None):
    if next_coroutine is not None:
        print(f"[filter_match_character] I send word with '{char}' char to the {id(next_coroutine)} coroutine")
    try:
        while True:
            word = yield
            if char in word:
                print(f"[filter_match_character] Found match for '{char}' in '{word}'")
                if next_coroutine:
                    next_coroutine.send(word)
    except GeneratorExit:
        print(f"[filter_match_character] Stopping filter for '{char}'")

def display_filtered_word():
    try:
        while True:
            word = yield
            print("[display_filtered_word] Printing in an amazing display...", word)
    except GeneratorExit:
        print("[display_filtered_word] Stopping amazing displayer")

display = display_filtered_word()
next(display)
filter_character_1 = filter_match_character(char="!", next_coroutine=display)
filter_character_1.__next__()
filter_character_2 = filter_match_character(char=",", next_coroutine=display)
filter_character_2.__next__()

word_producer("Hello, World!", [filter_character_1, filter_character_2])


[filter_match_character] I send word with '!' char to the 140529787670080 coroutine
[filter_match_character] I send word with ',' char to the 140529787670080 coroutine
[filter_match_character] Found match for ',' in 'Hello,'
[display_filtered_word] Printing in an amazing display... Hello,
[filter_match_character] Found match for '!' in 'World!'
[display_filtered_word] Printing in an amazing display... World!
[filter_match_character] Stopping filter for ','
